In [ ]:
#import pandas as pd
#from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)

#df_2020_2 = pd.DataFrame(client.get("rnvb-vnyh", limit=504872, offset=0))

#df_2020_1 = pd.DataFrame(client.get("a8xr-en99", limit=15435, offset=0))

In [ ]:
import polars as pl
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
# define functions
def lower_names(df):
    l_names = {col: col.lower() for col in df.columns}
    return df.rename(l_names)

def plot_null_vals(df):
    null_bool = df.select([pl.col(col).is_null() for col in df.columns])
    
    null_array = null_bool.to_numpy().astype(int).T
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.imshow(null_array, aspect='auto', cmap='viridis', interpolation='none')
    
    ax.set_yticks(range(len(df.columns)))
    ax.set_yticklabels(df.columns)
    ax.set_xticks([])
    plt.title('Valores nulos por columna')
    
    return plt.show()

def data_type(df):
    df_type = pl.DataFrame(
        {
            "col_name": df.columns,
            "var_type": [df_h[col].dtype for col in df.columns]
        }
    )

    return df_type

def impute_mode(df):
    
    for col in df.columns:
        n_row = df.shape[0]   
        mode_col = df[col].drop_nulls().mode().item()
        
        if df[col].null_count() > 0:
            df = df.with_columns(
                pl.col(col).fill_null(mode_col))
    return df

In [ ]:
# load data
df_h = pl.read_csv("/Users/sebas/OneDrive/Documents/repositorio_cepap/CEPAP/data_source/educacion/2025/Resultados__nicos_Saber_11_20250303.tsv", separator="\t", decimal_comma = True)

df_h.head()

In [ ]:
# clean names
df_h = lower_names(df_h)

df_h.head()

In [ ]:
# Explorar variables por prefijos
prefjs = ["cole", "estu_", "fami_", "punt_"]

In [ ]:
# Columnas colegio
columnas_cole = [col for col in df_h.columns if col.startswith(prefjs[0])]
df_h.select(columnas_cole).head()

In [ ]:
# Columnas estudiante
columnas_estu = [col for col in df_h.columns if col.startswith(prefjs[1])]
df_h.select(columnas_estu).head()

In [ ]:
# Columnas familia
columnas_fami = [col for col in df_h.columns if col.startswith(prefjs[2])]
df_h.select(columnas_fami).head()

In [ ]:
# Columnas puntaje
columnas_punt = [col for col in df_h.columns if col.startswith(prefjs[3])]
df_h.select(columnas_punt).head()

In [ ]:
# Conteo observaciones por periodo
df_h.group_by("periodo").len()

In [ ]:
# filter periods
df_h = df_h.filter(df_h["periodo"].is_in([20171, 20172, 20191, 20194]))

df_h.shape

In [ ]:
# sample 500.000 records
df_h = df_h.sample(n = 500000, seed = 2025)

In [ ]:
#clean data type
df_h = df_h.with_columns(
    pl.col("punt_matematicas").cast(pl.Float64),
    pl.col("punt_ingles").cast(pl.Float64)
)

In [ ]:
# print data type
data_type(df_h)

In [ ]:
cols_punt = [col for col in df_h.columns if col.startswith("punt_")]

df_h.select(cols_punt).describe()

In [ ]:
# Arbitrary select vars
df_h = df_h.select(["estu_genero", "cole_calendario", "fami_educacionmadre", "fami_educacionpadre",
                    "fami_estratovivienda", "fami_tienecomputador", "cole_area_ubicacion", "fami_tieneinternet", "punt_global"])

In [ ]:
# Conteo de la cantidad de valores perdidos y su proporción por columna
pl.DataFrame(
    {
        "col_name": df_h.columns,
        "missing_values": [df_h[col].null_count() for col in df_h.columns],
        "propotion_missing_values": [round((df_h[col].null_count() / df_h.shape[0]), 3) * 100 for col in df_h.columns]
    }
) \
.sort("missing_values", descending = True)

In [ ]:
plot_null_vals(df_h)

In [ ]:
df_h = impute_mode(df_h)

In [ ]:
# verfy delete null values
plot_null_vals(df_h)

In [ ]:
print("Luego de la selección arbitraria de variables y de la eliminación de datos perdidos, se observa un total de " + str(df_h.shape[0]) +
      " filas y " + str(df_h.shape[1]) + " columnas")

In [ ]:
# print data type
data_type(df_h)

In [ ]:
# plot count unique values
var_names_plot = df_h.columns

var_names_plot.remove("punt_global")

titles_plots = ["Cantidad de estudiantes desagregado por género", "Cantidad de estudianes desagregado por calendarios",
                "Cantidad de estudiantes desagregado por nivel educativo de la madre",
                "Cantidad de estudiantes desagregado por nivel educativo del padre",
                "Cantidad de estudiantes desagregados por estrato", "Cantidad de estudiantes que tienen o no tienen computador",
                "Cantidad de estudiantes desagregado por área de ubicación",
                "Cantidad de estudiantes con acceso o sin acceso a internet"]

for i in range(0, len(var_names_plot)):
    count_val = df_h[var_names_plot[i]].value_counts()

    plt.barh(count_val[var_names_plot[i]], count_val["count"])
    plt.xlabel("Frecuencia")
    plt.ylabel(var_names_plot[i])
    plt.title(titles_plots[i])
    plt.show()

plt.boxplot(df_h["punt_global"])
plt.title("Distribución de los puntajes globales de los estudiantes")
plt.show()
